In [2]:
# Source - https://stackoverflow.com/a
# Posted by Jared Sutton
# Retrieved 2025-11-23, License - CC BY-SA 4.0

import sys

if sys.version_info.major == 3 and sys.version_info.minor >= 10:
    import collections
    setattr(collections, "MutableMapping", collections.abc.MutableMapping)

In [3]:
import yfinance as yf

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA"]
info = yf.Tickers(" ".join(tickers))

live_data = {}
for t in tickers:
    live_data[t] = info.tickers[t].info.get("currentPrice")
print(live_data)

{'AAPL': 278.78, 'MSFT': 483.16, 'GOOGL': 321.27, 'AMZN': 229.53, 'NVDA': 182.41}


In [4]:
import pandas as pd
from typing import List, Dict

def download_1m(symbols: List[str], days: int) -> Dict[str, pd.Series]:
    """
    Download 1-minute close for symbols over N days.
    Returns dict: symbol -> pd.Series of close prices (may be empty).
    """
    out = {s: pd.Series(dtype=float) for s in symbols}
    try:
        df = yf.download(
            symbols,
            period=f"{days}d",
            interval="1m",
            group_by="ticker",
            auto_adjust=True,
            threads=True,
            progress=False,
            prepost=True,
        )
    except Exception:
        df = pd.DataFrame()

    for sym in symbols:
        try:
            if isinstance(df.columns, pd.MultiIndex):
                # Multi-ticker shape
                if (sym, "Close") in df.columns:
                    ser = df[(sym, "Close")].dropna()
                else:
                    ser = df[sym]["Close"].dropna()
            else:
                # Single-ticker shape
                ser = df["Close"].dropna() if "Close" in df.columns and len(symbols) == 1 else pd.Series(dtype=float)
        except Exception:
            ser = pd.Series(dtype=float)
        out[sym] = ser
    return out

out = download_1m(symbols = tickers, days = 1)

In [14]:
out

{'AAPL': Datetime
 2025-11-21 09:00:00+00:00    266.54
 2025-11-21 09:01:00+00:00    266.68
 2025-11-21 09:05:00+00:00    266.37
 2025-11-21 09:06:00+00:00    266.47
 2025-11-21 09:08:00+00:00    266.57
                               ...  
 2025-11-22 00:55:00+00:00    271.38
 2025-11-22 00:56:00+00:00    271.35
 2025-11-22 00:57:00+00:00    271.38
 2025-11-22 00:58:00+00:00    271.43
 2025-11-22 00:59:00+00:00    271.35
 Name: (AAPL, Close), Length: 919, dtype: float64,
 'MSFT': Datetime
 2025-11-21 09:00:00+00:00    478.3500
 2025-11-21 09:01:00+00:00    477.9600
 2025-11-21 09:02:00+00:00    478.1900
 2025-11-21 09:03:00+00:00    478.1900
 2025-11-21 09:04:00+00:00    478.1000
                                ...   
 2025-11-22 00:55:00+00:00    473.6012
 2025-11-22 00:56:00+00:00    473.6600
 2025-11-22 00:57:00+00:00    473.6000
 2025-11-22 00:58:00+00:00    473.5000
 2025-11-22 00:59:00+00:00    473.5700
 Name: (MSFT, Close), Length: 924, dtype: float64,
 'GOOGL': Datetime
 2025-1

In [30]:
from typing import Optional, Any

def safe_float(x):
    try:
        return float(x) if x is not None and not pd.isna(x) else None
    except Exception:
        return None

def get_fast_fields(t: yf.Ticker) -> Dict[str, Any]:
    fi = getattr(t, "fast_info", {}) or {}
    return {
        "last_price": safe_float(fi.get("lastPrice")),
        "previous_close": safe_float(fi.get("previousClose")),
        "open": safe_float(fi.get("open")),
        "day_high": safe_float(fi.get("dayHigh")),
        "day_low": safe_float(fi.get("dayLow")),
        "volume": safe_float(fi.get("lastVolume") or fi.get("volume")),
        "market_cap": safe_float(fi.get("marketCap")),
        "currency": fi.get("currency"),
    }

def get_name_field(t: yf.Ticker) -> Optional[str]:
    # Try the lighter get_info first, then fallback to info
    try:
        info = t.get_info(retry_count=1, timeout=5.0) or {}
        return info.get("shortName") or info.get("longName")
    except Exception:
        try:
            info = t.info or {}
            return info.get("shortName") or info.get("longName")
        except Exception:
            return None

def build_row(sym: str, today_1m: pd.Series, five_1m: pd.Series) -> Dict[str, Any]:
    t = yf.Ticker(sym)
    fq = get_fast_fields(t)
    name = get_name_field(t)

    # Sparkline source: today if available, else last available from 5d
    spark = today_1m if today_1m is not None and len(today_1m) else five_1m
    if spark is None or len(spark) == 0:
        sparkline = []
        last_price = fq["last_price"]
        last_ts = None
    else:
        # Trim to a manageable number of points for the UI (e.g., last 240 minutes)
        if len(spark) > 240:
            spark = spark.iloc[-240:]
        sparkline = [safe_float(v) for v in spark.values.tolist()]
        last_price = safe_float(spark.iloc[-1])
        last_ts = spark.index[-1].isoformat()

    price = last_price or fq["last_price"]
    prev_close = fq["previous_close"]

    change = None
    change_pct = None
    if price is not None and prev_close not in (None, 0):
        change = price - prev_close
        change_pct = (change / prev_close) * 100.0

    return {
        "ticker": sym,
        "name": name,
        "price": None if price is None else round(price, 4),
        "change": None if change is None else round(change, 4),
        "change_pct": None if change_pct is None else round(change_pct, 3),
        "prev_close": None if prev_close is None else round(prev_close, 4),
        "open": None if fq["open"] is None else round(fq["open"], 4),
        "day_high": None if fq["day_high"] is None else round(fq["day_high"], 4),
        "day_low": None if fq["day_low"] is None else round(fq["day_low"], 4),
        "volume": None if fq["volume"] is None else int(fq["volume"]),
        "market_cap": None if fq["market_cap"] is None else int(fq["market_cap"]),
        "currency": fq["currency"],
        "sparkline": sparkline,      # list of floats for your tiny chart
        "points": len(sparkline),    # FYI
        "last_point_ts": last_ts,    # ISO timestamp of last datapoint
    }


In [26]:
build_row('AAPL', out['AAPL'], None)

lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']


{'ticker': 'AAPL',
 'name': 'Apple Inc.',
 'price': 271.35,
 'change': 5.24,
 'change_pct': 1.969,
 'prev_close': 266.11,
 'open': 265.95,
 'day_high': 273.33,
 'day_low': 265.67,
 'volume': 58784100,
 'market_cap': 4029017336174,
 'currency': 'USD',
 'sparkline': [271.19000244140625,
  270.760009765625,
  270.739990234375,
  271.4150085449219,
  270.7349853515625,
  270.7099914550781,
  270.7300109863281,
  270.9949951171875,
  271.489990234375,
  271.0,
  271.07,
  271.08,
  271.08,
  271.0,
  270.99,
  271.09,
  270.87,
  270.801,
  270.7551,
  270.77,
  270.7156,
  270.7,
  270.8727,
  270.88,
  270.74,
  270.75,
  270.77,
  270.83,
  270.7265,
  270.7159,
  270.75,
  270.6999,
  270.7154,
  270.71,
  270.71,
  270.8295,
  271.11,
  271.13,
  271.2,
  270.76,
  270.87,
  270.99,
  270.99,
  270.79,
  270.84,
  270.8587,
  270.925,
  270.93,
  270.89,
  270.93,
  270.9,
  270.9,
  270.9593,
  270.9109,
  270.75,
  270.72,
  270.7761,
  270.9,
  271.2,
  270.82,
  270.85,
  270.8,
  

In [40]:
sp_100_tickers = pd.read_csv("data/sp100_tickers.csv")
sp100_data = []

tickers = sp_100_tickers['Symbol'].apply(lambda x: x.replace(".", '-')).to_list()

# fetch the 1m_data
toady_1m_data = download_1m(symbols=tickers, days=1)

for ticker in tickers:
    sp100_data.append(build_row(ticker, toady_1m_data[ticker], None))

In [43]:
print(sp100_data[3])

{'ticker': 'ACN', 'name': 'Accenture plc', 'price': 252.25, 'change': 11.46, 'change_pct': 4.759, 'prev_close': 240.79, 'open': 241.88, 'day_high': 253.97, 'day_low': 241.29, 'volume': 5896900, 'market_cap': 156865455541, 'currency': 'USD', 'sparkline': [253.2550048828125, 253.0220947265625, 252.960693359375, 252.77999877929688, 252.89999389648438, 252.97000122070312, 252.88999938964844, 252.76499938964844, 252.5800018310547, 252.88999938964844, 252.75, 252.9199981689453, 252.6699981689453, 252.69000244140625, 252.80999755859375, 252.8300018310547, 252.98500061035156, 252.94500732421875, 252.83999633789062, 252.5399932861328, 252.5800018310547, 252.11500549316406, 252.52000427246094, 252.72000122070312, 253.0, 253.07000732421875, 253.13999938964844, 253.26499938964844, 253.1699981689453, 253.49000549316406, 253.52000427246094, 253.5850067138672, 253.4199981689453, 253.27999877929688, 253.2100067138672, 253.2100067138672, 253.25, 253.25999450683594, 253.16000366210938, 252.75, 252.85499

In [44]:
import json

data = {
    "sp100": sp100_data
}

with open("data/sp100_live_data", "w") as f:
    json.dump(data, f)

get 1 day data to show graph

In [ ]:
data = download_1m(symbols=['AAPL'], days=2)
last_date = data['AAPL'].index.date[-1]
last_day = data['AAPL'][data['AAPL'].index.date == last_date]
print(last_day)

Datetime
2025-12-06 00:00:00+00:00    279.0300
2025-12-06 00:01:00+00:00    279.0300
2025-12-06 00:02:00+00:00    279.1300
2025-12-06 00:03:00+00:00    279.1299
2025-12-06 00:04:00+00:00    279.1200
2025-12-06 00:05:00+00:00    279.0800
2025-12-06 00:06:00+00:00    279.1189
2025-12-06 00:07:00+00:00    279.1200
2025-12-06 00:08:00+00:00    279.1200
2025-12-06 00:09:00+00:00    279.1199
2025-12-06 00:10:00+00:00    279.0700
2025-12-06 00:11:00+00:00    279.0700
2025-12-06 00:12:00+00:00    279.1191
2025-12-06 00:13:00+00:00    279.0600
2025-12-06 00:14:00+00:00    279.0892
2025-12-06 00:15:00+00:00    279.0601
2025-12-06 00:16:00+00:00    279.1100
2025-12-06 00:17:00+00:00    279.1185
2025-12-06 00:18:00+00:00    279.0900
2025-12-06 00:19:00+00:00    279.0800
2025-12-06 00:20:00+00:00    279.1200
2025-12-06 00:21:00+00:00    279.0800
2025-12-06 00:22:00+00:00    279.1200
2025-12-06 00:23:00+00:00    279.1000
2025-12-06 00:24:00+00:00    279.1000
2025-12-06 00:25:00+00:00    279.1000
202

: 